In [1]:
import utils.helper_functions as hf

import numpy as np
import requests
import json
import time
import pandas as pd
import mysql.connector
from IPython.display import clear_output
from datetime import datetime, timedelta
from config import api_football_key, conn_host, conn_database, conn_user, conn_password
import os
from termcolor import colored
import pickle
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from dateutil import tz
from joblib import load
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_winner(home_score, away_score):
    if home_score > away_score:
        return 'H'
    elif away_score > home_score:
        return 'A'
    else:
        return 'D'

In [3]:
leagues = [
    {'league_id': 40, 'country': 'england', 'league': 'championship'},
    {'league_id': 39, 'country': 'england', 'league': 'premier-league'},
    {'league_id': 78, 'country': 'germany', 'league': 'bundesliga'},
#     {'league_id': 61, 'country': 'france', 'league': 'ligue-1', 'min_threshold': 1.95},
#     {'league_id': 140, 'country': 'spain', 'league': 'laliga'},
#     {'league_id': 71, 'country': 'brazil', 'league': 'serie-a'},
#     {'league_id': 72, 'country': 'brazil', 'league': 'serie-b'},
#     {'league_id': 253, 'country': 'usa', 'league': 'mls'},
#     {'league_id': 128, 'country': 'argentina', 'league': 'liga-profesional'},
#     {'league_id': 98, 'country': 'japan', 'league': 'j1-league'},
]

In [4]:
current_season = 2022

league_selected_index = 0
league_id, country, league = leagues[league_selected_index]['league_id'], leagues[league_selected_index]['country'], leagues[league_selected_index]['league']

n_last_games = 5
now = time.time()
now_datetime = datetime.today()

min_threshold = leagues[league_selected_index].get('min_threshold') or 2
max_threshold = 10

draw_threshold = 0
min_score_diff_threshold = 0.4

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Sao_Paulo')

headers = {
        'X-RapidAPI-Key': api_football_key,
        'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
    }

In [5]:
teams = hf.execute_query("SELECT * FROM teams")

In [6]:
def get_league_season_fixtures(season):
    response = requests.get(f"https://api-football-v1.p.rapidapi.com/v3/fixtures?league={league_id}&season={season}", headers=headers)
    response_parsed = json.loads(response.text)
    return response_parsed['response']

In [7]:
def add_match_info_to_db(fixture):
    fixture_id, league_id, league_name, fixture_date, season, home_id, home_name, away_id, away_name, home_score, away_score = fixture['fixture']['id'], fixture['league']['id'], f"{fixture['league']['name']} ({fixture['league']['country']})", datetime.utcfromtimestamp(fixture['fixture']['timestamp']).replace(tzinfo=from_zone).astimezone(to_zone), fixture['league']['season'], fixture['teams']['home']['id'], fixture['teams']['home']['name'], fixture['teams']['away']['id'], fixture['teams']['away']['name'], fixture['goals']['home'], fixture['goals']['away']
    fixture_date_converted = fixture_date.strftime('%Y-%m-%d %H:%M:%S')
    hf.execute_query(f"INSERT IGNORE INTO leagues (id, name) VALUES ({league_id}, '{league_name}')", False)
    hf.execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({home_id}, '{home_name}')", False)
    hf.execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({away_id}, '{away_name}')", False)
    hf.execute_query(f"INSERT IGNORE INTO matches (id, date, league_id, season, home_id, away_id, home_score, away_score) VALUES ({fixture_id}, '{fixture_date_converted}', {league_id}, {season}, {home_id}, {away_id}, '{home_score}', '{away_score}')", False)

In [8]:
for season in range(current_season, current_season+1):
    fixtures_resp = get_league_season_fixtures(season)
    fixtures = [fixture for fixture in fixtures_resp if fixture['fixture']['timestamp'] < now]
    if season == current_season:
        next_fixtures = [fixture for fixture in fixtures_resp if fixture['fixture']['timestamp'] >= now and fixture['fixture']['timestamp'] <= (now + 26*60*60)]
        next_fixtures = sorted(next_fixtures, key = lambda x: x['fixture']['timestamp'])
        
    for index, fixture in enumerate(fixtures):
        clear_output(wait=True)
        print(f"Loading fixtures for the {season} season: {index}/{len(fixtures)}")
        add_match_info_to_db(fixture)

Loading fixtures for the 2022 season: 404/405


In [9]:
query = f"select m.id from matches as m where m.league_id = {league_id} and m.season = {current_season} and m.home_passes_pct is null and m.no_stats is null"
missing_values_fixtures = hf.execute_query(query)

In [10]:
missing_values_fixtures.tail()

,id


In [11]:
def get_league_season_fixtures(fixture_id):
    response = requests.get(f"https://api-football-v1.p.rapidapi.com/v3/fixtures/statistics?fixture={fixture_id}", headers=headers)
    response_parsed = json.loads(response.text)
    return response_parsed['response']

def update_match_stats(fixture_id, home_stats, away_stats):
    home_update_query = f"SET home_shots_on_goal = {home_stats[0] or 0}, home_shots_off_goal = {home_stats[1] or 0}, home_total_shots = {home_stats[2] or 0}, home_blocked_shots = {home_stats[3] or 0}, home_shots_inside_box = {home_stats[4] or 0}, home_shots_outside_box = {home_stats[5] or 0}, home_fouls = {home_stats[6] or 0}, home_corners = {home_stats[7] or 0}, home_offsides = {home_stats[8] or 0}, home_possession = {home_stats[9].replace('%', '')}, home_yellow_cards = {home_stats[10] or 0}, home_red_cards = {home_stats[11] or 0}, home_saves = {home_stats[12] or 0}, home_total_passes = {home_stats[13] or 0}, home_passes_accurate = {home_stats[14] or 0}, home_passes_pct = {home_stats[15].replace('%', '')}"
    away_update_query = f"away_shots_on_goal = {away_stats[0] or 0}, away_shots_off_goal = {away_stats[1] or 0}, away_total_shots = {away_stats[2] or 0}, away_blocked_shots = {away_stats[3] or 0}, away_shots_inside_box = {away_stats[4] or 0}, away_shots_outside_box = {away_stats[5] or 0}, away_fouls = {away_stats[6] or 0}, away_corners = {away_stats[7] or 0}, away_offsides = {away_stats[8] or 0}, away_possession = {away_stats[9].replace('%', '')}, away_yellow_cards = {away_stats[10] or 0}, away_red_cards = {away_stats[11] or 0}, away_saves = {away_stats[12] or 0}, away_total_passes = {away_stats[13] or 0}, away_passes_accurate = {away_stats[14] or 0}, away_passes_pct = {away_stats[15].replace('%', '')}"
    update_query = f"UPDATE matches {home_update_query}, {away_update_query} WHERE id = {fixture_id}"
    return update_query

def update_match_no_stats(fixture_id):
    update_query = f"UPDATE matches SET no_stats = 1 WHERE id = {fixture_id}"
    return update_query

def get_fixtures_statistics(fixtures):
    update_queries = []
    
    for index, g in fixtures.iterrows():
        clear_output(wait=True)
        print(f"{index}/{len(fixtures.index)}")
        
        try:
            stats = get_league_season_fixtures(g['id'])
            home_stats = [s['value'] for s in stats[0]['statistics']]
            away_stats = [s['value'] for s in stats[1]['statistics']]
            query = update_match_stats(g['id'], home_stats, away_stats)
            update_queries.append(query)
        except Exception as e:
            query = update_match_no_stats(g['id'])

        update_queries.append(query)
        
    hf.execute_multiple_queries(update_queries)

In [12]:
get_fixtures_statistics(missing_values_fixtures)

In [13]:
query = f"SELECT m.id, m.date, m.season, l.name AS league, ht.id as home_id, at.id as away_id, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds, " + \
                            "m.home_shots_on_goal, m.home_shots_off_goal, m.home_total_shots, m.home_blocked_shots, m.home_shots_inside_box, m.home_shots_outside_box, m.home_fouls, m.home_corners, m.home_offsides, m.home_possession, m.home_yellow_cards, m.home_red_cards, m.home_saves, m.home_total_passes, m.home_passes_accurate, m.home_passes_pct, " + \
                            "m.away_shots_on_goal, m.away_shots_off_goal, m.away_total_shots, m.away_blocked_shots, m.away_shots_inside_box, m.away_shots_outside_box, m.away_fouls, m.away_corners, m.away_offsides, m.away_possession, m.away_yellow_cards, m.away_red_cards, m.away_saves, m.away_total_passes, m.away_passes_accurate, m.away_passes_pct " + \
                            f"FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE (l.id = {league_id} AND m.season = {current_season}) ORDER BY m.date ASC"
fixtures_df = hf.execute_query(query)
fixtures_df['winner'] = fixtures_df.apply(lambda x: get_winner(x['home_score'], x['away_score']), axis=1)

In [14]:
fixtures_df.head()

,id,date,season,league,home_id,away_id,home_team,away_team,home_score,away_score,...,away_corners,away_offsides,away_possession,away_yellow_cards,away_red_cards,away_saves,away_total_passes,away_passes_accurate,away_passes_pct,winner
0,874671,2022-07-29 16:00:00,2022,Championship (England),37,44,Huddersfield,Burnley,0,1,...,5.0,3.0,69.0,2.0,0.0,0.0,599.0,506.0,84.0,A
1,874672,2022-07-30 11:00:00,2022,Championship (England),67,72,Blackburn,QPR,1,0,...,0.0,3.0,50.0,1.0,0.0,3.0,529.0,430.0,81.0,H
2,874673,2022-07-30 11:00:00,2022,Championship (England),1356,53,Blackpool,Reading,1,0,...,10.0,1.0,52.0,0.0,0.0,4.0,422.0,321.0,76.0,H
3,874674,2022-07-30 11:00:00,2022,Championship (England),43,71,Cardiff,Norwich,1,0,...,7.0,2.0,55.0,2.0,1.0,1.0,432.0,336.0,78.0,H
4,874675,2022-07-30 11:00:00,2022,Championship (England),64,56,Hull,Bristol City,2,1,...,3.0,4.0,45.0,0.0,0.0,2.0,382.0,306.0,80.0,H


In [15]:
model = load(f"leagues/{league_id}/model.joblib")

In [17]:
for fixture in next_fixtures:
    fixture_id, league_id, league_name, fixture_date, season, home_id, home_name, away_id, away_name, home_score, away_score = fixture['fixture']['id'], fixture['league']['id'], f"{fixture['league']['name']} ({fixture['league']['country']})", fixture['fixture']['timestamp'], fixture['league']['season'], fixture['teams']['home']['id'], fixture['teams']['home']['name'], fixture['teams']['away']['id'], fixture['teams']['away']['name'], fixture['goals']['home'], fixture['goals']['away']
    fixture_date_converted = datetime.fromtimestamp(fixture_date)
    
    home_stats = hf.get_team_previous_games_stats(home_id, season, fixture_date_converted, 'H', n_last_games, fixtures_df)
    if home_stats == None:
        continue
        
    away_stats = hf.get_team_previous_games_stats(away_id, season, fixture_date_converted, 'A', n_last_games, fixtures_df)
    if away_stats == None:
        continue
    
    game_stats = [home_stats + away_stats]
    
    predictions = model.predict(game_stats)
    
    home_score_pred, away_score_pred = predictions[0]
    
    prediction = 'D'
    prediction_text = 'Draw'
    score_pred_diff = abs(home_score_pred - away_score_pred)
    if home_score_pred > away_score_pred and score_pred_diff > draw_threshold:
        prediction = 'H'
        prediction_text = home_name
    elif home_score_pred < away_score_pred and score_pred_diff > draw_threshold:
        prediction = 'A'
        prediction_text = away_name
        
    print('-'*10)
    print(f"{home_name} x {away_name}")
    if score_pred_diff > draw_threshold and score_pred_diff < min_score_diff_threshold:
        print(colored(f"BAD BET: {prediction_text} ({round(home_score_pred, 2)} x {round(away_score_pred, 2)})\n", 'red'))
    else:
        odds = float(input(f"Type the odds for a {prediction_text} result: ")) 
        if selected_odds < min_threshold or selected_odds > max_threshold:
            ml_good_bet_conditions_text = f"Good bet if {min_threshold} < odds < {max_threshold}"
            print(colored(f"BAD BET: {prediction_text} @ {selected_odds} ({ml_good_bet_conditions_text})\n", 'red'))
        else:
            print(colored(f'GOOD BET: {prediction_text} @ {selected_odds}\n', 'green'))
        

----------
Hull City x West Brom
BAD BET: West Brom (1.19 x 1.23)

